In [8]:
import pandas as pd

# Load the data
orders = pd.read_csv('orders.csv')
regions = pd.read_csv('regions.csv')

# Merge the datasets
data = pd.merge(orders, regions, on='Territory')

# Convert 'OrderDate' and 'DeliveryDate' to datetime
data['OrderDate'] = pd.to_datetime(data['OrderDate'])
data['DeliveryDate'] = pd.to_datetime(data['DeliveryDate'])

# Normalize quantities: split the 'Quantities' string and convert to integers, then sum them
data['TotalQuantities'] = data['Quantities'].apply(lambda x: sum(int(q) for q in x.split(',')))

# Calculate the delivery time in days before the aggregation
data['DeliveryTimeDays'] = (data['DeliveryDate'] - data['OrderDate']).dt.days

# Extract year and assign seasons
data['Year'] = data['OrderDate'].dt.year
data['Season'] = data['OrderDate'].dt.month.apply(lambda x: 'Winter' if x <= 2 or x is 12 else ('Spring' if x <= 5 else ('Summer' if x <= 8 else 'Autumn')))

# Group by year and season
grouped = data.groupby(['Year', 'Season'])

# Aggregate data using pre-calculated 'DeliveryTimeDays' and 'TotalQuantities'
summary = grouped.agg(
    total_revenue=pd.NamedAgg(column='CartPriceInCP', aggfunc='sum'),
    total_orders=pd.NamedAgg(column='OrderID', aggfunc='count'),
    total_products=pd.NamedAgg(column='TotalQuantities', aggfunc='sum'),
    average_delivery_time=pd.NamedAgg(column='DeliveryTimeDays', aggfunc='mean')
)

# Convert to JSON
summary_json = summary.reset_index().to_json(orient='records')

# Write JSON to a file
with open('summary_data.json', 'w') as file:
    file.write(summary_json)


<>:22: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:22: SyntaxWarning: "is" with a literal. Did you mean "=="?
/tmp/ipykernel_4502/1751361845.py:22: SyntaxWarning: "is" with a literal. Did you mean "=="?
  data['Season'] = data['OrderDate'].dt.month.apply(lambda x: 'Winter' if x <= 2 or x is 12 else ('Spring' if x <= 5 else ('Summer' if x <= 8 else 'Autumn')))
